<a href="https://colab.research.google.com/github/MassimilianoBiancucci/Tensorflow-exercises/blob/master/MLPs/networksfully_connected_deep_networks_using_tox21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
np.random.seed(456)
import  tensorflow as tf
tf.set_random_seed(456)
import matplotlib.pyplot as plt
import deepchem as dc
from sklearn.metrics import accuracy_score